In [1]:
from mecode import GMatrix
from helpers.gmatrix_extensions import get_gmatrix, pen_down, pen_up


# Draw a unit

In [3]:
def draw_unit(g: GMatrix, path: list[list[tuple[float, float]]]) -> None:
    """Draw a unit of the repeating pattern. This method assumes the pen is raised at the starting position when called.

    Args:
        g (GMatrix): GMatrix instance to use for GCode generation.
        path (list[list[tuple[float, float]]]): List of strokes to draw. Each stroke is a list of points. The coordinates are relative to the current position.
    """
    
    # Set to absolute coordinates
    is_relative = g.is_relative
    if is_relative:
        g.absolute()

    # Convert path in relative coordinates to absolute coordinates, based on the current position
    path = path.copy() # Important: make a copy of the path, so that the original path is not modified
    current_x = g.current_position['x']
    current_y = g.current_position['y']
    for i, stroke in enumerate(path):
        path[i] = [(x + current_x, y + current_y)
                   for x, y in stroke]

    for stroke in path:
        # Move to the first point of the stroke and start drawing
        x0, y0 = stroke[0]
        g.abs_move(x0, y0)
        pen_down(g)

        # Draw the stroke by moving to each point
        for x, y in stroke[1:]:
            g.abs_move(x, y)

        # Finish drawing the stroke. Raise the pen to prepare for the next stroke.
        pen_up(g)

    # Restore the coordinate mode
    if is_relative:
        g.relative()

In [4]:
square_pattern = [
    [(0, 10), (0, 0), (10, 0)]  # The left and top sides of the square
]  # This pattern contains only one stroke.

rotated_square_pattern = [
    [(0, 0), (20, 20)],  # diagonal top-left to bottom-right
    [(20, 15), (15, 10), (20, 5)],  # right triangle
    [(20, 0), (0, 20)],  # diagonal top-right to bottom-left
    [(0, 15), (5, 10), (0, 5)],  # left triangle
    [(5, 0), (10, 5), (15, 0)],  # top triangle
    [(5, 20), (10, 15), (15, 20)],  # bottom triangle
]


In [5]:
# Unit test for draw_unit
g: GMatrix = get_gmatrix("unit_test.gcode")
pen_up(g)
draw_unit(g, rotated_square_pattern)


# Draw repeating pattern

In [6]:
def draw_reapeating_pattern(
        g: GMatrix,
        width: float,
        height: float,
        path: list[list[tuple[float, float]]],
        scale: float,
        repeat: tuple[int, int]
        ) -> None:
    """Draw a unit of the repeating pattern. This method assumes the pen is raised at the starting position when called.

    Args:
        g (GMatrix): GMatrix instance to use for GCode generation.
        width (float): Width of the pattern in mm. This is the distance moved in x-direction after each pattern is drawn.
        height (float): Height of the pattern in mm. This is the distance moved in y-direction after each line of patterns is drawn.
        path (list[list[tuple[float, float]]]): List of strokes to draw. Each stroke is a list of points.
        scale (float): Scale factor to apply to the pattern.
        repeat (tuple[int, int]): Number of times to repeat the pattern in the x and y directions, respectively.
    """

    # Check if there is any point in path that is outside the limit of width and height
    for stroke in path:
        for x, y in stroke:
            if x > width or y > height:
                raise ValueError("Path is outside the dimension limit")

    # Scale the pattern
    path = [[(x * scale, y * scale) for x, y in stroke] for stroke in path]
    width = width * scale
    height = height * scale

    # Check if the repeating pattern takes more space than the page
    # Page limit: 205x275 mm
    if width * repeat[0] > 205 or height * repeat[1] > 275:
        raise ValueError("Repeating pattern is outside the dimension limit")

    # Set to absolute mode for drawing repeating patterns
    g.absolute()

    # Draw the pattern from the current position
    x_initial = g.current_position['x']
    y_initial = g.current_position['y']
    x = x_initial
    y = y_initial
    for y_rep in range(repeat[1]):

        # Draw each row of the pattern
        for x_rep in range(repeat[0]):
            draw_unit(g, path)
            x += width
            g.abs_move(x, y)
        
        # Move to the next row
        x = x_initial
        y += height
        g.abs_move(x_initial, y)

    # Set to relative mode
    g.relative()

In [8]:
# Useful GCode snippets
end_code = ""

# Initialise GCode generation and write start code
g: GMatrix = get_gmatrix("output.gcode")

# Move the pen to a reasonable starting position
pen_up(g)
g.move(30, 30)

# Draw the pattern
draw_reapeating_pattern(g, 20, 20, rotated_square_pattern, 1, (4, 5))

# Write end code
g.pop_matrix()
g.write(end_code)
